In [1]:
#Include Libraries:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [2]:
#Preparing Dataset
dataset_dir = "./Dataset/"
chotot_dataset = pd.read_csv(dataset_dir + "chotot.csv",index_col = [0])
chotot_dataset.head()

,Price,Area,Deposit Fee,Furniture,Ward,District
0,1600000,20,500000.0,NaN,Hòa Thọ Đông,Cẩm Lệ
1,1100000,40,50000.0,Nội thất đầy đủ,Thuận Phước,Hải Châu
2,1500000,20,500000.0,Nhà trống,Hòa Phát,Cẩm Lệ
3,2500000,25,2500000.0,Nội thất đầy đủ,Hòa Minh,Liên Chiểu
4,1200000,30,1000000.0,Nội thất đầy đủ,Thuận Phước,Hải Châu


In [3]:
phongtro123_dataset = pd.read_csv(dataset_dir + "phongtro123.csv",index_col=[0])
phongtro123_dataset.head()

,Price,Area,Ward,District
0,1300000.0,20,Hòa Minh,Liên Chiểu
1,3000000.0,25,Nại Hiên Đông,Sơn Trà
2,1500000.0,15,Hòa Khê,Thanh Khê
3,2600000.0,30,An Hải Bắc,Sơn Trà
4,3200000.0,40,Hòa Khánh Nam,Liên Chiểu


In [4]:
chotot_dataset = chotot_dataset.drop(columns=['Furniture','Deposit Fee'])
Dataset = pd.concat([chotot_dataset, phongtro123_dataset])
print(Dataset.shape)
Dataset.head()

(1654, 4)


,Price,Area,Ward,District
0,1600000.0,20,Hòa Thọ Đông,Cẩm Lệ
1,1100000.0,40,Thuận Phước,Hải Châu
2,1500000.0,20,Hòa Phát,Cẩm Lệ
3,2500000.0,25,Hòa Minh,Liên Chiểu
4,1200000.0,30,Thuận Phước,Hải Châu


In [5]:
Dataset.isna().sum()

Price        20
Area          0
Ward        104
District     36
dtype: int64

array([' Cẩm Lệ', ' Hải Châu', ' Liên Chiểu', ' Sơn Trà', ' Ngũ Hành Sơn',
       ' Thanh Khê', ' Hòa Vang', ' Hoà Vang', ' Thanh Quan', nan],
      dtype=object)

In [7]:
Dataset['Price'] = Dataset.fillna(value = Dataset['Price'].mean(), inplace = True)
Dataset['Ward'] = 

SyntaxError: invalid syntax (3932935114.py, line 2)